# 🌾 Crop Prediction Model Training
## Deep Learning Approach with Feature Engineering

**Goal:** Train a neural network to predict the best crop based on soil and climate parameters

**Input Features (7):**
- N (Nitrogen)
- P (Phosphorus)
- K (Potassium)
- Temperature
- Humidity
- pH
- Rainfall

**Output:** Crop recommendation with confidence scores

**Dataset:** https://www.kaggle.com/datasets/atharvaingle/crop-recommendation-dataset

## 📦 Step 1: Import Libraries

In [ ]:
# ============================================
# FIX: Protobuf compatibility for Kaggle
# ============================================
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TensorFlow warnings

# ============================================
# Import Libraries
# ============================================

# Data manipulation
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
try:
    plt.style.use('seaborn-v0_8-darkgrid')
except:
    plt.style.use('seaborn-darkgrid')

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    accuracy_score,
    top_k_accuracy_score
)

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import (
    EarlyStopping, 
    ModelCheckpoint, 
    ReduceLROnPlateau
)
from tensorflow.keras.utils import to_categorical

# Utilities
import joblib
import json
from datetime import datetime

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Verify setup
print(f"✅ TensorFlow version: {tf.__version__}")
print(f"✅ Pandas version: {pd.__version__}")
print(f"✅ NumPy version: {np.__version__}")
print(f"✅ GPU available: {len(tf.config.list_physical_devices('GPU')) > 0}")

if len(tf.config.list_physical_devices('GPU')) > 0:
    print(f"✅ Using GPU for training (faster)")
else:
    print(f"⚠️  Using CPU for training (slower but reliable)")

## 📥 Step 2: Load Dataset

In [ ]:
# Load dataset
# For Kaggle: Use /kaggle/input/crop-recommendation-dataset/Crop_recommendation.csv
# For local: Use your local path

# Kaggle path (uncomment when running on Kaggle)
df = pd.read_csv('/kaggle/input/crop-recommendation-dataset/Crop_recommendation.csv')

# Local path (uncomment for local testing)
# df = pd.read_csv('datasets/crop_data/Crop_recommendation.csv')

print(f"📊 Dataset shape: {df.shape}")
print(f"\n📋 First 5 rows:")
df.head()

In [ ]:
# Dataset info
print("📊 Dataset Information:")
print(f"\nTotal samples: {len(df)}")
print(f"Total crops: {df['label'].nunique()}")
print(f"\nCrop distribution:")
print(df['label'].value_counts())

print(f"\n📈 Basic statistics:")
df.describe()

In [ ]:
# Check for missing values
print("🔍 Missing values:")
print(df.isnull().sum())

# Check data types
print("\n📋 Data types:")
print(df.dtypes)

## 📊 Step 3: Exploratory Data Analysis (EDA)

In [ ]:
# Visualize crop distribution
plt.figure(figsize=(14, 6))
crop_counts = df['label'].value_counts()
plt.bar(crop_counts.index, crop_counts.values, color='skyblue', edgecolor='black')
plt.xlabel('Crop', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.title('Distribution of Crops in Dataset', fontsize=14, fontweight='bold')
plt.xticks(rotation=90)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"✅ Dataset is {'balanced' if crop_counts.std() < 10 else 'imbalanced'}")

In [ ]:
# Feature distributions
features = ['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for i, feature in enumerate(features):
    axes[i].hist(df[feature], bins=30, color='skyblue', edgecolor='black', alpha=0.7)
    axes[i].set_title(f'{feature} Distribution', fontweight='bold')
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel('Frequency')
    axes[i].grid(alpha=0.3)

# Remove extra subplot
fig.delaxes(axes[7])

plt.tight_layout()
plt.show()

In [ ]:
# Correlation heatmap
plt.figure(figsize=(10, 8))
correlation_matrix = df[features].corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Heatmap', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 🔧 Step 4: Feature Engineering

Transform 7 input features into 15 engineered features for better model performance.

In [ ]:
def engineer_features(df):
    """
    Engineer features from raw inputs
    7 features → 15 features
    """
    df_engineered = df.copy()
    
    # Original features
    N = df_engineered['N']
    P = df_engineered['P']
    K = df_engineered['K']
    temp = df_engineered['temperature']
    humidity = df_engineered['humidity']
    ph = df_engineered['ph']
    rainfall = df_engineered['rainfall']
    
    # Engineered features
    df_engineered['NPK_sum'] = N + P + K
    df_engineered['N_P_ratio'] = N / (P + 0.01)
    df_engineered['N_K_ratio'] = N / (K + 0.01)
    df_engineered['P_K_ratio'] = P / (K + 0.01)
    df_engineered['temp_humidity'] = temp * humidity / 100
    df_engineered['rainfall_humidity'] = rainfall * humidity / 100
    
    # Categorical encoding
    df_engineered['temp_category'] = pd.cut(temp, 
                                             bins=[-np.inf, 15, 25, 35, np.inf], 
                                             labels=[0, 1, 2, 3])
    df_engineered['rainfall_category'] = pd.cut(rainfall, 
                                                 bins=[-np.inf, 100, 200, 300, np.inf], 
                                                 labels=[0, 1, 2, 3])
    
    return df_engineered

# Apply feature engineering
df_engineered = engineer_features(df)

print("✅ Feature engineering completed!")
print(f"\nOriginal features: 7")
print(f"Engineered features: {len(df_engineered.columns) - 1}")
print(f"\nNew columns added:")
print([col for col in df_engineered.columns if col not in df.columns])

df_engineered.head()

## 📊 Step 5: Prepare Data for Training

In [ ]:
# Separate features and target
feature_columns = [
    'N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall',
    'NPK_sum', 'N_P_ratio', 'N_K_ratio', 'P_K_ratio',
    'temp_humidity', 'rainfall_humidity',
    'temp_category', 'rainfall_category'
]

X = df_engineered[feature_columns].values
y = df_engineered['label'].values

print(f"✅ Features shape: {X.shape}")
print(f"✅ Target shape: {y.shape}")

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
num_classes = len(label_encoder.classes_)

print(f"✅ Number of classes: {num_classes}")
print(f"✅ Crop names: {list(label_encoder.classes_)}")

In [ ]:
# Split data: 80% train, 10% val, 10% test
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y_encoded, test_size=0.1, random_state=42, stratify=y_encoded
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.111, random_state=42, stratify=y_temp
)

print(f"✅ Train samples: {len(X_train)} ({len(X_train)/len(X)*100:.1f}%)")
print(f"✅ Validation samples: {len(X_val)} ({len(X_val)/len(X)*100:.1f}%)")
print(f"✅ Test samples: {len(X_test)} ({len(X_test)/len(X)*100:.1f}%)")

In [ ]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("✅ Features scaled (StandardScaler)")
print(f"   Mean: {X_train_scaled.mean():.4f}")
print(f"   Std: {X_train_scaled.std():.4f}")

In [ ]:
# One-hot encode labels
y_train_cat = to_categorical(y_train, num_classes)
y_val_cat = to_categorical(y_val, num_classes)
y_test_cat = to_categorical(y_test, num_classes)

print(f"✅ Labels one-hot encoded")
print(f"   Shape: {y_train_cat.shape}")

## 🏗️ Step 6: Build Deep Learning Model

In [ ]:
def build_model(input_dim, num_classes):
    """
    Build Deep Learning model for crop prediction
    Architecture: 15 → 256 → 128 → 64 → num_classes
    """
    model = Sequential([
        # Input layer
        Dense(256, activation='relu', input_shape=(input_dim,)),
        Dropout(0.3),
        
        # Hidden layer 1
        Dense(128, activation='relu'),
        Dropout(0.3),
        
        # Hidden layer 2
        Dense(64, activation='relu'),
        Dropout(0.2),
        
        # Output layer
        Dense(num_classes, activation='softmax')
    ])
    
    # Compile model
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=[
            'accuracy',
            keras.metrics.TopKCategoricalAccuracy(k=3, name='top_3_accuracy')
        ]
    )
    
    return model

# Build model
model = build_model(input_dim=15, num_classes=num_classes)

print("✅ Model built successfully!")
model.summary()

## 🎯 Step 7: Train Model

In [ ]:
# Define callbacks
callbacks = [
    # Early stopping
    EarlyStopping(
        monitor='val_accuracy',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    
    # Model checkpoint
    ModelCheckpoint(
        'best_crop_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    
    # Reduce learning rate on plateau
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=0.00001,
        verbose=1
    )
]

print("✅ Callbacks configured")

In [ ]:
# Train model
print("🎯 Starting training...\n")

history = model.fit(
    X_train_scaled, y_train_cat,
    validation_data=(X_val_scaled, y_val_cat),
    epochs=100,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

print("\n✅ Training completed!")

## 📈 Step 8: Visualize Training History

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Accuracy
axes[0].plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
axes[0].plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
axes[0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Top-3 Accuracy
axes[1].plot(history.history['top_3_accuracy'], label='Train Top-3', linewidth=2)
axes[1].plot(history.history['val_top_3_accuracy'], label='Val Top-3', linewidth=2)
axes[1].set_title('Top-3 Accuracy', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Top-3 Accuracy')
axes[1].legend()
axes[1].grid(alpha=0.3)

# Loss
axes[2].plot(history.history['loss'], label='Train Loss', linewidth=2)
axes[2].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
axes[2].set_title('Model Loss', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('Loss')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 🧪 Step 9: Evaluate Model

In [ ]:
# Evaluate on test set
test_loss, test_accuracy, test_top3_accuracy = model.evaluate(
    X_test_scaled, y_test_cat, verbose=0
)

print("="*60)
print("📊 TEST SET RESULTS")
print("="*60)
print(f"Test Accuracy: {test_accuracy*100:.2f}%")
print(f"Test Top-3 Accuracy: {test_top3_accuracy*100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")
print("="*60)

In [ ]:
# Predictions
y_pred_proba = model.predict(X_test_scaled, verbose=0)
y_pred = np.argmax(y_pred_proba, axis=1)

# Classification report
print("\n📊 Classification Report:\n")
print(classification_report(
    y_test, y_pred, 
    target_names=label_encoder.classes_,
    zero_division=0
))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(14, 12))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_,
            cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix', fontsize=16, fontweight='bold')
plt.xlabel('Predicted Crop', fontsize=12)
plt.ylabel('Actual Crop', fontsize=12)
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

## 🧪 Step 10: Test with Sample Predictions

In [ ]:
def predict_crop(features_dict, model, scaler, label_encoder):
    """
    Predict crop from input features
    """
    # Extract features
    N = features_dict['nitrogen']
    P = features_dict['phosphorus']
    K = features_dict['potassium']
    temp = features_dict['temperature']
    humidity = features_dict['humidity']
    ph = features_dict['ph_value']
    rainfall = features_dict['rainfall']
    
    # Engineer features (MUST MATCH TRAINING!)
    NPK_sum = N + P + K
    N_P_ratio = N / (P + 0.01)
    N_K_ratio = N / (K + 0.01)
    P_K_ratio = P / (K + 0.01)
    temp_humidity = temp * humidity / 100
    rainfall_humidity = rainfall * humidity / 100
    
    # Categorical encoding (same bins as training)
    if temp < 15:
        temp_cat = 0
    elif temp < 25:
        temp_cat = 1
    elif temp < 35:
        temp_cat = 2
    else:
        temp_cat = 3
    
    if rainfall < 100:
        rain_cat = 0
    elif rainfall < 200:
        rain_cat = 1
    elif rainfall < 300:
        rain_cat = 2
    else:
        rain_cat = 3
    
    # Create feature array (EXACT ORDER AS TRAINING)
    features = np.array([[
        N, P, K, temp, humidity, ph, rainfall,
        NPK_sum, N_P_ratio, N_K_ratio, P_K_ratio,
        temp_humidity, rainfall_humidity,
        float(temp_cat), float(rain_cat)  # Convert to float
    ]])
    
    # Scale features
    X_scaled = scaler.transform(features)
    
    # Predict
    proba = model.predict(X_scaled, verbose=0)[0]
    
    # Get top 3
    top3_idx = np.argsort(proba)[-3:][::-1]
    top3_crops = label_encoder.inverse_transform(top3_idx)
    top3_conf = proba[top3_idx]
    
    return {
        'predicted_crop': top3_crops[0].capitalize(),
        'confidence': top3_conf[0] * 100,
        'top_3': [(crop.capitalize(), conf * 100) for crop, conf in zip(top3_crops, top3_conf)]
    }

# Test cases
test_cases = [
    {
        'name': 'Rice',
        'nitrogen': 90, 'phosphorus': 42, 'potassium': 43,
        'temperature': 21, 'humidity': 82, 'ph_value': 6.5, 'rainfall': 203
    },
    {
        'name': 'Wheat',
        'nitrogen': 80, 'phosphorus': 40, 'potassium': 40,
        'temperature': 18, 'humidity': 65, 'ph_value': 6.8, 'rainfall': 600
    },
    {
        'name': 'Maize',
        'nitrogen': 90, 'phosphorus': 40, 'potassium': 40,
        'temperature': 21, 'humidity': 82, 'ph_value': 6.5, 'rainfall': 100
    },
    {
        'name': 'Coffee',
        'nitrogen': 100, 'phosphorus': 50, 'potassium': 35,
        'temperature': 25, 'humidity': 85, 'ph_value': 6.8, 'rainfall': 200
    },
    {
        'name': 'Cotton',
        'nitrogen': 120, 'phosphorus': 50, 'potassium': 50,
        'temperature': 25, 'humidity': 70, 'ph_value': 7.0, 'rainfall': 100
    },
    {
        'name': 'Apple',
        'nitrogen': 20, 'phosphorus': 125, 'potassium': 200,
        'temperature': 22, 'humidity': 85, 'ph_value': 6.5, 'rainfall': 120
    },
    {
        'name': 'Banana',
        'nitrogen': 100, 'phosphorus': 80, 'potassium': 50,
        'temperature': 28, 'humidity': 85, 'ph_value': 6.5, 'rainfall': 150
    },
    {
        'name': 'Chickpea',
        'nitrogen': 40, 'phosphorus': 60, 'potassium': 80,
        'temperature': 25, 'humidity': 60, 'ph_value': 7.0, 'rainfall': 60
    }
]

print("🧪 Testing Sample Predictions:\n")
print("="*70)

correct = 0
total = len(test_cases)

for test in test_cases:
    name = test.pop('name')
    result = predict_crop(test, model, scaler, label_encoder)
    
    print(f"\n🌾 Expected: {name}")
    print(f"   Predicted: {result['predicted_crop']} ({result['confidence']:.1f}%)")
    print(f"   Top-3:")
    for i, (crop, conf) in enumerate(result['top_3'], 1):
        print(f"      {i}. {crop:15s} {conf:5.1f}%")
    
    if result['predicted_crop'].lower() == name.lower():
        print("   ✅ CORRECT!")
        correct += 1
    else:
        top3_names = [c[0].lower() for c in result['top_3']]
        if name.lower() in top3_names:
            print("   ⚠️  In Top-3 but not #1")
        else:
            print("   ❌ Not in Top-3")

print("\n" + "="*70)
print(f"📊 Test Results: {correct}/{total} correct ({correct/total*100:.1f}%)")
print("="*70)

## 💾 Step 11: Save Model and Artifacts

In [ ]:
# Save model
model.save('best_crop_model.h5')
print("✅ Saved: best_crop_model.h5")

# Save scaler
joblib.dump(scaler, 'scaler.pkl')
print("✅ Saved: scaler.pkl")

# Save label encoder
joblib.dump(label_encoder, 'label_encoder.pkl')
print("✅ Saved: label_encoder.pkl")

# Save metadata
metadata = {
    'model_type': 'MLP',
    'model_version': '1.0',
    'training_date': datetime.now().isoformat(),
    'num_features': 15,
    'num_classes': num_classes,
    'feature_names': feature_columns,
    'class_names': label_encoder.classes_.tolist(),
    'test_accuracy': float(test_accuracy),
    'test_top3_accuracy': float(test_top3_accuracy),
    'test_loss': float(test_loss),
    'training_samples': len(X_train),
    'validation_samples': len(X_val),
    'test_samples': len(X_test),
    'epochs_trained': len(history.history['loss']),
    'best_val_accuracy': float(max(history.history['val_accuracy'])),
    'architecture': {
        'input': 15,
        'dense_1': 256,
        'dense_2': 128,
        'dense_3': 64,
        'output': num_classes
    }
}

with open('metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)
print("✅ Saved: metadata.json")

print("\n🎉 All artifacts saved successfully!")

## 📊 Step 12: Final Summary

In [ ]:
print("\n" + "="*70)
print("🎉 CROP PREDICTION MODEL TRAINING COMPLETED!")
print("="*70)
print(f"\n📊 Final Results:")
print(f"   Test Accuracy: {test_accuracy*100:.2f}%")
print(f"   Top-3 Accuracy: {test_top3_accuracy*100:.2f}%")
print(f"   Total Crops: {num_classes}")
print(f"   Total Parameters: {model.count_params():,}")
print(f"   Training Epochs: {len(history.history['loss'])}")
print(f"\n📦 Saved Files:")
print(f"   - best_crop_model.h5 (Model)")
print(f"   - scaler.pkl (Feature Scaler)")
print(f"   - label_encoder.pkl (Label Encoder)")
print(f"   - metadata.json (Training Info)")
print(f"\n🎯 Next Steps:")
print(f"   1. Download the model files")
print(f"   2. Place in Django project: ml_models/")
print(f"   3. Run: python manage.py runserver")
print(f"   4. Test at: http://127.0.0.1:8000/predictions/crop/")
print("\n" + "="*70)